In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import jieba 
from gensim import *
%matplotlib inline

d:\program files\python36\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [ ]:
# set parameters:
vocab_dim = 200
maxlen = 100
n_iterations = 1  # ideally more..
n_exposures = 10
window_size = 7
batch_size = 32
n_epoch = 4
input_length = 100
cpu_count = multiprocessing.cpu_count()

In [2]:
data=pd.read_csv('data/train_first.csv')
data=data.drop(['Id'],axis=1)
data.head()

,Discuss,Score
0,好大的一个游乐公园，已经去了2次，但感觉还没有玩够似的！会有第三，第四次的,5
1,新中国成立也是在这举行，对我们中国人来说有些重要及深刻的意义！,4
2,庐山瀑布非常有名，也有非常多个瀑布，只是最好看的非三叠泉莫属，推荐一去,4
3,个人觉得颐和园是北京最值的一起的地方，不过相比下门票也是最贵的，比起故宫的雄伟与气势磅礴，颐...,5
4,迪斯尼一日游,5


In [3]:
#统计查看
#plt.figure()
#plt.plot(data[:100].groupby('Score').count()[0],data[:100].groupby('Score').count()[1])
#plt.show()

In [4]:
#加载停用词
def get_stopwords(path):
    return [line.strip() for line in open(path,'r',encoding='utf-8').readlines()]
#句子去停用词
def removestopwords(sentence):
        stopwords_list=get_stopwords('data/stopwords.txt')
        outstr=''
        for word in sentence:
            if not word in stopwords_list:
                if word!='\n' and word!='\t':
                     outstr+=word+' '
        return outstr

In [5]:
#分词 并去掉停用词
def cut(sentence):
    return removestopwords(jieba.cut(sentence))
data['Discuss']=data['Discuss'].map(cut)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ZHANGG~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.175 seconds.
Prefix dict has been built succesfully.


In [ ]:
#word2vec

#创建词语字典，并返回每个词语的索引，词向量，以及每个句子所对应的词语索引
def word2vec_train(combined):

    model = Word2Vec(size=vocab_dim,
                     min_count=n_exposures,
                     window=window_size,
                     workers=cpu_count,
                     iter=n_iterations)
    model.build_vocab(combined)
    model.train(combined)
    model.save('data/Word2vec_model.pkl')
    index_dict, word_vectors,combined = create_dictionaries(model=model,combined=combined)
    return   index_dict, word_vectors,combined


#创建词语字典，并返回每个词语的索引，词向量，以及每个句子所对应的词语索引
def create_dictionaries(model=None,
                        combined=None):
    ''' Function does are number of Jobs:
        1- Creates a word to index mapping
        2- Creates a word to vector mapping
        3- Transforms the Training and Testing Dictionaries

    '''
    if (combined is not None) and (model is not None):
        gensim_dict = Dictionary()
        gensim_dict.doc2bow(model.vocab.keys(),
                            allow_update=True)
        w2indx = {v: k+1 for k, v in gensim_dict.items()}#所有频数超过10的词语的索引
        w2vec = {word: model[word] for word in w2indx.keys()}#所有频数超过10的词语的词向量

        def parse_dataset(combined):
            ''' Words become integers
            '''
            data=[]
            for sentence in combined:
                new_txt = []
                for word in sentence:
                    try:
                        new_txt.append(w2indx[word])
                    except:
                        new_txt.append(0)
                data.append(new_txt)
            return data
        combined=parse_dataset(combined)
        combined= sequence.pad_sequences(combined, maxlen=maxlen)#每个句子所含词语对应的索引，所以句子中含有频数小于10的词语，索引为0
        return w2indx, w2vec,combined
    else:
        print ('No data provided...')

In [ ]:
#搭建网络结构

In [ ]:
#训练

In [ ]:
#预测